# Notebook to capture results of ICA/NMF factor clustering experiments

In [ ]:
import matplotlib.pyplot as plt
import sys
%matplotlib inline

In [ ]:
sys.path += ['../Src']

In [ ]:
import importlib
import factor_clustering
importlib.reload(factor_clustering)
from factor_clustering import FactorClustering
from factorizer_wrappers import NMF_Factorizer, ICA_Factorizer

## Angles of random vectors in high dimensions
In high dimension spaces, any two random vectors are close to 90 degrees to each other!

In [ ]:
FactorClustering.demonstrate_angles_in_high_dimensions()
plt.show()

## Perform multiple repeat calculations of ICA and NMF and look at clustering

In [ ]:
expression_file = '../Data/HGSOC_Protein_Expression.csv'

fc = FactorClustering()

fc.read_expression_matrix(expression_file)

In [ ]:
n_repeats = 50
if True:
    # Beware - this will take hours (for the full size dataset)!
    fc.compute_and_cache_multiple_factor_repeats(2, 21, n_repeats, force=False)

In [ ]:
if True:
    fc.plot_multiple_combined_factors_scatter(2, 14, n_repeats)

In [ ]:
if True:
    fc.find_best_n_components(NMF_Factorizer, 2, 21, 50, doprint=False, doshow=True)  
    fc.find_best_n_components(ICA_Factorizer, 2, 21, 50, doprint=False, doshow=True)  

In [ ]:
if True:
    fc.plot_multiple_single_factors_scatter(NMF_Factorizer, 2, 14, 50)

In [ ]:
if True:
    fc.plot_multiple_single_factors_scatter(ICA_Factorizer, 2, 14, 50)